In [3]:
from dl.data_loader import import_data
from dl.data_loader import preprocess
import numpy as np
from sklearn.preprocessing import StandardScaler

In [5]:
import os
os.getcwd()

'C:\\Users\\Thomas\\Desktop\\DeepLearningProject\\deeplearning'